# 1. Get S&P data from Yahoo Finance

In [3]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import os
import pandas_ta
import warnings

warnings.filterwarnings('ignore')

# Define a file path to save and load the data
data_file = 'stock_data.csv'

# Check if the data file exists, and load it if it does
if os.path.exists(data_file):
    df = pd.read_csv(data_file, index_col=[0, 1], header=0, parse_dates=True)
else:
    # If the data file doesn't exist, download the data
    sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

    # symbols cleanup - some symbols contain dots ".", we need to replace them with "-" so we can read data with yfinance
    sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

    # grab list with all the stock symbols
    symbols_list = sp500['Symbol'].unique().tolist()

    # Remove 'VLTO' from the list - it does not have enough data for later calculations
    symbols_list.remove('VLTO')

    end_date = '2023-10-31'
    start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8) # 8 years ago

    # download data from yahooFinance
    df = yf.download(tickers=symbols_list,
      start=start_date,
      end=end_date).stack()


    # Save the downloaded data to a file
    df.to_csv(data_file)


# add/change column names
df.index.names = ['date', 'ticker']
# fix column names to lower case
df.columns = df.columns.str.lower()

df

adj close       close        high         low        open  \
date       ticker                                                               
2015-11-02 A        36.128700   38.590000   38.619999   37.799999   37.869999   
           AAL      44.313286   46.470001   46.820000   46.200001   46.200001   
           AAPL     27.498981   30.295000   30.340000   29.902500   30.200001   
           ABBV     44.862259   63.380001   64.199997   61.439999   61.599998   
           ABT      39.108421   45.430000   45.500000   44.599998   44.880001   
...                       ...         ...         ...         ...         ...   
2023-10-30 YUM     119.870003  119.870003  120.639999  119.260002  120.290001   
           ZBH     103.410004  103.410004  104.099998  102.330002  103.760002   
           ZBRA    209.770004  209.770004  211.210007  204.470001  207.500000   
           ZION     29.980000   29.980000   30.180000   29.320000   29.840000   
           ZTS     156.625000  157.000000  158.410004  153.520004  157.130005   

                        volume  
date       ticker               
2015-11-02 A         1810800.0  
           AAL       6189300.0  
           AAPL    128813200.0  
           ABBV     17008700.0  
           ABT       5477800.0  
...                        ...  
2023-10-30 YUM       1551900.0  
           ZBH       1309800.0  
           ZBRA       970000.0  
           ZION      2837100.0  
           ZTS       2756200.0  

[984474 rows x 6 columns]

# 2. Calculate features and technical indicators for each stock
- Garman-Klass Volatility (intraday volatility for a given asset)
- RSI
- Bollinger Bands
- ATR
- MACD
- Dollar Volume

Garman-Klass Volatility
\begin{equation}
\frac{(ln(High) - ln(Low))^{2}}{2}-(2ln(2)-1)(ln(\text{Adj Close})-ln(Open))^{2}
\end{equation}



In [4]:
## Garman-Klass Volatility
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2)/2-(2*np.log(2)-1)*(np.log(df['adj close'])-np.log(df['open']))**2

## RSI - groupby (levl=1) == ticker
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

## Bollinger Bands - low - mid - high
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

## ATR - we need to define custom function as it needs three columns and it would f-up the data otherwise
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                    low=stock_data['low'],
                    close=stock_data['close'],
                    length=14)
    return atr.sub(atr.mean()).div(atr.std())
df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

## MACD - same logic as with the ATR indicator
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length= 20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

## Dollar Volume
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

adj close       close        high         low        open  \
date       ticker                                                               
2015-11-02 A        36.128700   38.590000   38.619999   37.799999   37.869999   
           AAL      44.313286   46.470001   46.820000   46.200001   46.200001   
           AAPL     27.498981   30.295000   30.340000   29.902500   30.200001   
           ABBV     44.862259   63.380001   64.199997   61.439999   61.599998   
           ABT      39.108421   45.430000   45.500000   44.599998   44.880001   
...                       ...         ...         ...         ...         ...   
2023-10-30 YUM     119.870003  119.870003  120.639999  119.260002  120.290001   
           ZBH     103.410004  103.410004  104.099998  102.330002  103.760002   
           ZBRA    209.770004  209.770004  211.210007  204.470001  207.500000   
           ZION     29.980000   29.980000   30.180000   29.320000   29.840000   
           ZTS     156.625000  157.000000  158.410004  153.520004  157.130005   

                        volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-11-02 A         1810800.0         -0.000626        NaN       NaN   
           AAL       6189300.0         -0.000583        NaN       NaN   
           AAPL    128813200.0         -0.003286        NaN       NaN   
           ABBV     17008700.0         -0.037869        NaN       NaN   
           ABT       5477800.0         -0.007120        NaN       NaN   
...                        ...               ...        ...       ...   
2023-10-30 YUM       1551900.0          0.000061  43.110984  4.765199   
           ZBH       1309800.0          0.000143  33.314329  4.620582   
           ZBRA       970000.0          0.000480  38.278362  5.283422   
           ZION      2837100.0          0.000409  39.613464  3.376202   
           ZTS       2756200.0          0.000488  27.979275  5.059724   

                     bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-11-02 A            NaN       NaN       NaN       NaN      65.421850  
           AAL          NaN       NaN       NaN       NaN     274.268220  
           AAPL         NaN       NaN       NaN       NaN    3542.231801  
           ABBV         NaN       NaN       NaN       NaN     763.048703  
           ABT          NaN       NaN       NaN       NaN     214.228110  
...                     ...       ...       ...       ...            ...  
2023-10-30 YUM     4.793638  4.822076  0.292694 -1.214160     186.026257  
           ZBH     4.675267  4.729951 -0.336426 -1.339901     135.446423  
           ZBRA    5.366760  5.450097 -0.096500 -1.273832     203.476904  
           ZION    3.520557  3.664912  0.159876 -1.244780      85.056257  
           ZTS     5.134859  5.209994  0.629278 -2.575641     431.689825  

[984474 rows x 14 columns]

# 3. Aggregate to monthly level and filter top 150 most liquid stocks for each month
- To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency

In [33]:
## define last columns that wont feature any value model will use
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
          df.unstack()[last_cols].resample('M').last().stack('ticker')],
          axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2015-12-31 A          103.333141   39.250359         -0.001536  58.471772   
           AAL        319.493873   40.471584         -0.000806  45.226549   
           AAPL      4264.137789   23.988562         -0.004376  33.365717   
           ABBV       294.510523   41.931839         -0.047227  51.321208   
           ABT        211.476885   38.660786         -0.009106  48.706197   
...                          ...         ...               ...        ...   
2023-10-31 YUM        197.474648  119.870003          0.000061  43.110984   
           ZBH        169.430979  103.410004          0.000143  33.314329   
           ZBRA       106.996075  209.770004          0.000480  38.278362   
           ZION       117.608804   29.980000          0.000409  39.613464   
           ZTS        272.115930  156.625000          0.000488  27.979275   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2015-12-31 A       3.644514  3.676936  3.709358 -1.133284  0.254618  
           AAL     3.695287  3.743884  3.792481  0.537265 -0.050796  
           AAPL    3.195609  3.270628  3.345647 -0.983822 -0.588078  
           ABBV    3.671835  3.723142  3.774448 -1.031209 -0.331810  
           ABT     3.656644  3.682610  3.708575 -1.040552 -0.179479  
...                     ...       ...       ...       ...       ...  
2023-10-31 YUM     4.765199  4.793638  4.822076  0.292694 -1.214160  
           ZBH     4.620582  4.675267  4.729951 -0.336426 -1.339901  
           ZBRA    5.283422  5.366760  5.450097 -0.096500 -1.273832  
           ZION    3.376202  3.520557  3.664912  0.159876 -1.244780  
           ZTS     5.059724  5.134859  5.209994  0.629278 -2.575641  

[46480 rows x 9 columns]

- Calculate 5-year rolling average of dollar volume for each stocks before filtering

In [37]:
#data['dollar_volume'] = (data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())

# data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

## we can drop dollar volume and dollar_vol_rank since we got out top 150 stocks
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2020-11-30 AAL      14.130000          0.001401  58.332321  2.440051   
           AAPL    117.138092          0.000611  54.008295  4.700143   
           ABBV     92.321548         -0.006367  71.866466  4.373911   
           ABT     102.758224         -0.000784  48.978009  4.621924   
           ACN     238.940948         -0.000525  61.931867  5.376573   
...                       ...               ...        ...       ...   
2023-10-31 VRTX    357.450012          0.000045  49.528779  5.855406   
           VZ       34.619999          0.000191  60.808388  3.418412   
           WFC      39.430000          0.000237  43.426276  3.664971   
           WMT     163.020004          0.000080  55.462094  5.056755   
           XOM     105.879997          0.000065  40.895808  4.658489   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2020-11-30 AAL     2.613901  2.787751 -0.811768  0.861395  
           AAPL    4.752488  4.804832  0.684875 -0.073307  
           ABBV    4.471928  4.569946  0.061689  2.118375  
           ABT     4.663147  4.704369  1.028727 -0.258669  
           ACN     5.443973  5.511373  0.242796  1.254769  
...                     ...       ...       ...       ...  
2023-10-31 VRTX    5.895537  5.935668  0.446444  0.300403  
           VZ      3.494735  3.571058  0.202399  0.985819  
           WFC     3.711807  3.758643 -0.510405 -0.554081  
           WMT     5.082165  5.107575  0.290557  0.015461  
           XOM     4.705436  4.752383  1.189158 -1.470715  

[5364 rows x 8 columns]